In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import numpy as np
import glob
import gc
print(datetime.datetime.now())
os.getcwd()

2019-10-17 13:57:32.521423


'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Slice_ids_FB_Rewards_offers'

In [2]:
files_Q3_audience_uploaded=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/upload_files/*.csv")
print(len(files_Q3_audience_uploaded))

50


In [3]:
files_control_regular=[x for x in files_Q3_audience_uploaded if os.path.basename(x)[:2]=="C_"]
print(len(files_control_regular))
list_dfs=[]
for file in files_control_regular:
    df=pd.read_csv(file,dtype=str)
    list_dfs.append(df)
df_all_Q3_control=pd.concat(list_dfs)

25


In [4]:
print(df_all_Q3_control.shape)
df_all_Q3_control.head(3)

(542702, 3)


,customer_id_hashed,email_address_hash,segment
0,8679c14bdd15650f9778cce34e009f334471e5c28499f8...,b1b635b4fdde83bc119badacf451de238a789da94e57b1...,C_Quadrant III_P_H_2019Q3_RFM
1,7e2af687c741f9a48249c83815e51bfcff4c2909954e40...,9215f3daaedc6c4c86393bb3422dd1a24ff063e7b7cacc...,C_Quadrant III_P_H_2019Q3_RFM
2,ba0ecd1d9b58f20207e939a13578134fb528ea1dcf7ba1...,4ae4a97859e8908afe21171d0eecfc07bb0934d78a9b7b...,C_Quadrant III_P_H_2019Q3_RFM


In [5]:
last_Saturday=datetime.date(2019,10,9)
date_1_year_ago=last_Saturday-datetime.timedelta(days=365)
print(date_1_year_ago)

dates_18_months_ago=last_Saturday-datetime.timedelta(days=365*1.5)
print(dates_18_months_ago)

2018-10-09
2018-04-10


In [6]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

# Shoppers_in_18_months

In [7]:
All_item_sales_files=list(recursive_file_gen("/home/jian/BigLots/"))
All_item_sales_files=[x for x in All_item_sales_files if "2019" in x or "2018" in x]
All_item_sales_files=[x for x in All_item_sales_files if "2017" not in x]
All_item_sales_files=[x for x in All_item_sales_files if "subclasslevel" not in x]
All_item_sales_files=[x for x in All_item_sales_files if "daily" in x.lower() and x[-4:]==".txt"]
All_item_sales_files.sort()
len(All_item_sales_files)

All_item_sales_files_18=[]
for file in All_item_sales_files:
    if "item_transaction_amt" in pd.read_table(file,dtype=str,nrows=5,sep="|").columns.tolist():
        All_item_sales_files_18.append(file)
All_item_sales_files_18.sort()   

# Since 20180811

In [8]:
All_sub_sales_files_1=list(recursive_file_gen("/home/jian/BigLots/hist_daily_data_subclasslevel/"))
All_sub_sales_files_1=[x for x in All_sub_sales_files_1 if x.split("MediaStormDailySales_week_ending_")[1][:10]<"2018-08-11"]
All_sub_sales_files_1=[x for x in All_sub_sales_files_1 if x.split("MediaStormDailySales_week_ending_")[1][:10]>=str(dates_18_months_ago)]
All_sub_sales_files_1=[x for x in All_sub_sales_files_1 if "daily" in x.lower() and x[-4:]==".txt"]
All_sub_sales_files_1.sort()
print(len(All_sub_sales_files_1))

All_sub_sales_files_2=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
All_sub_sales_files_2=[x for x in All_sub_sales_files_2 if "daily" in x.lower() and x[-4:]==".txt"]
All_sub_sales_files_2=[x for x in All_sub_sales_files_2 if x.split("/MediaStorm_")[1][:10]<"2018-08-11"]
All_sub_sales_files_2=[x for x in All_sub_sales_files_2 if x.split("/MediaStorm_")[1][:10]>=str(dates_18_months_ago)]
All_sub_sales_files_2.sort()
print(len(All_sub_sales_files_2))
All_sub_sales_files=All_sub_sales_files_1+All_sub_sales_files_2
print(len(All_sub_sales_files))

9
8
17


In [10]:
list_all_daily_files=All_sub_sales_files+All_item_sales_files_18
print(len(list_all_daily_files))
print(52*1.5)

79
78.0


In [11]:
df_ids_shopped_in_18=pd.DataFrame()
list_df_shoppers=[]
i_counter=0
for file in list_all_daily_files:
    df=pd.read_table(file,usecols=['customer_id_hashed','transaction_dt'],sep="|",dtype=str)
    df=df[pd.notnull(df['customer_id_hashed'])]
    df=df[df['transaction_dt']>=str(dates_18_months_ago)]
    df=df.drop_duplicates()
    list_df_shoppers.append(df)
    i_counter+=1
    if i_counter%10==1:
        print(i_counter,datetime.datetime.now())
df_ids_shopped_in_18=pd.concat(list_df_shoppers)

1 2019-10-17 10:20:42.205337
11 2019-10-17 10:41:19.956638
21 2019-10-17 11:07:43.250180
31 2019-10-17 11:35:46.069065
41 2019-10-17 12:21:16.934975
51 2019-10-17 12:36:08.966259
61 2019-10-17 12:42:20.069750
71 2019-10-17 12:48:14.010567


In [12]:
print(df_ids_shopped_in_18.shape)
df_ids_shopped_in_18.head(2)

(105513593, 2)


,transaction_dt,customer_id_hashed
2,2018-04-11,8590d87322d534cb0c9b7f5d10f91ef8722fe4d6b3c367...
6,2018-04-10,ca47ec26ff378339e3266f1f48eb88809f90fd184161e4...


In [13]:
del list_df_shoppers
gc.collect()

633

In [14]:
df_ids_shopped_in_18=df_ids_shopped_in_18.sort_values(['transaction_dt','customer_id_hashed'],ascending=[False,True])
df_ids_shopped_in_18=df_ids_shopped_in_18.drop_duplicates("customer_id_hashed")
df_ids_shopped_in_18.shape

(21715881, 2)

In [17]:
max_date=df_ids_shopped_in_18['transaction_dt'].max()
min_date=df_ids_shopped_in_18['transaction_dt'].min()

print(min_date,max_date)

df_ids_shopped_in_18.to_csv("./BL_df_all_ids_shopped_in_18_months_"+min_date+"_"+max_date+"_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

2018-04-10 2019-10-12


In [9]:
df_ids_shopped_in_18=pd.read_csv("./BL_df_all_ids_shopped_in_18_months_2018-04-10_2019-10-12_JL_2019-10-17.csv",dtype=str)
df_ids_shopped_in_18.shape

(21715881, 2)

In [10]:
df_ids_shopped_in_18.head(2)

,transaction_dt,customer_id_hashed
0,2019-10-12,000001dadc0265bf9d250566d74e0006323f18b5826641...
1,2019-10-12,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...


In [11]:
print(df_ids_shopped_in_18['transaction_dt'].apply(len).unique())
print(df_ids_shopped_in_18['customer_id_hashed'].apply(len).unique())

[10]
[64]


# Signed_up_in_12_Months

In [12]:
list_all_master_file_2019=list(recursive_file_gen("/home/jian/BigLots/"))
list_all_master_file_2019=[x for x in list_all_master_file_2019 if "2019" in x and "aster" in x and x[-4:]==".txt"]
print(len(list_all_master_file_2019))
list_all_master_file_2019.sort()

40


In [13]:
df_signed_19=pd.DataFrame()
for file in list_all_master_file_2019:
    df=pd.read_csv(file,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_date'])
    df_signed_19=df_signed_19.append(df)
print(df_signed_19.shape)
df_signed_19=df_signed_19[df_signed_19['sign_up_date']>=str(date_1_year_ago)] 
print(df_signed_19.shape)

(4379752, 3)
(4379752, 3)


In [14]:
df_singed_18Q1_19Q1=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                                 compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_date'])
print(df_singed_18Q1_19Q1.shape)
df_singed_18Q1_19Q1=df_singed_18Q1_19Q1[df_singed_18Q1_19Q1['sign_up_date']>=str(date_1_year_ago)]
print(df_singed_18Q1_19Q1.shape)

df_singed_19Q1_gap=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                                 compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_date'])
print(df_singed_19Q1_gap.shape)
df_singed_19Q1_gap=df_singed_19Q1_gap[df_singed_19Q1_gap['sign_up_date']>=str(date_1_year_ago)]
print(df_singed_19Q1_gap.shape)


(7888562, 3)
(4199575, 3)
(461199, 3)
(461199, 3)


In [15]:
df_new_signed_12months=df_signed_19.append(df_singed_18Q1_19Q1).append(df_singed_19Q1_gap)
del df_signed_19
del df_singed_18Q1_19Q1
del df_singed_19Q1_gap
gc.collect()

1757

In [16]:
print(df_new_signed_12months.shape)
df_new_signed_12months=df_new_signed_12months[df_new_signed_12months['sign_up_date']>=str(date_1_year_ago)]
print(df_new_signed_12months.shape)

del df_new_signed_12months['sign_up_date']
df_new_signed_12months=df_new_signed_12months.drop_duplicates()
print(df_new_signed_12months.shape)


(9040526, 3)
(9040526, 3)
(7042900, 2)


In [17]:
df_new_signed_12months.head(2)

,customer_id_hashed,email_address_hash
0,9b7f9a1ba51dbae2a393c273ce8c9aa9bf610dc03410fd...,089bc373eda16a771e023d87ee368fb961c4ff9b36c7d6...
1,c2c5bd862113c4e77c918dd22a25744f2e1224ea63b166...,065edf59d0ed26bea352be5f44fd7a6f688a0fa28cddb3...


In [18]:
df_new_signed_12months['customer_id_hashed'].nunique()

7042111

In [19]:
df_new_signed_12months['email_address_hash'].nunique()

7042752

# Mapping_ID_with_email

In [32]:
df_singed_18Q1_19Q1=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                                  compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_date'])


df_singed_19Q1_gap=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                                 compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_date'])

df_singed_lapsed=pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip',
                     dtype = 'str',sep = '|',usecols=['customer_id_hashed','email_address_hash','sign_up_date'])

In [34]:
dfiddetail=pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/combined_masterids_up_to_20181229_JL.csv',usecols=['customer_id_hashed','email_address_hash'])
dfiddetail['sign_up_date']=np.nan

In [35]:
list_all_master_file=list(recursive_file_gen("/home/jian/BigLots/"))
list_all_master_file=[x for x in list_all_master_file if "aster" in x and x[-4:]==".txt"]
list_all_master_file.sort()

In [36]:
list_all_master=[]

for file in list_all_master_file:
    df=pd.read_csv(file,dtype = 'str',sep = '|',usecols=['customer_id_hashed','email_address_hash','sign_up_date'])
    list_all_master.append(df)
df_all_master=pd.concat(list_all_master)

In [37]:
df_all_master=df_all_master.append(df_singed_18Q1_19Q1).append(df_singed_19Q1_gap).append(df_singed_lapsed).append(dfiddetail)

del df_singed_18Q1_19Q1
del df_singed_19Q1_gap
del df_singed_lapsed
del dfiddetail
gc.collect()

392

In [38]:
df_all_master=df_all_master.sort_values(["customer_id_hashed","sign_up_date"],ascending=[True,False])
df_all_master=df_all_master[['customer_id_hashed','email_address_hash']].drop_duplicates()
gc.collect()
df_all_master.shape

(30751705, 2)

In [39]:
df_all_master['customer_id_hashed'].nunique(),df_all_master['email_address_hash'].nunique()

(30732612, 30744012)

In [40]:
df_all_master=df_all_master.drop_duplicates("customer_id_hashed")
df_all_master.shape

(30732612, 2)

In [43]:
df_all_master.head(2)

,customer_id_hashed,email_address_hash
13968893,00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...,4a1f9ca814426d0168b4ee9ac037be04151dff5631d78a...
126959,000001dadc0265bf9d250566d74e0006323f18b5826641...,26454de1f315ee0c13c8c9033f372f50a3f02a16691849...


In [45]:
df_ids_shopped_in_18=pd.merge(df_ids_shopped_in_18,df_all_master,on="customer_id_hashed",how="left")
df_ids_shopped_in_18.shape

(21715881, 3)

In [54]:
df_new_signed_12months=df_new_signed_12months.drop_duplicates("customer_id_hashed")

del df_ids_shopped_in_18['transaction_dt']
df_ids_shopped_in_18=df_ids_shopped_in_18.drop_duplicates("customer_id_hashed")
gc.collect()


df_both=df_new_signed_12months.append(df_ids_shopped_in_18)

df_both=df_both.drop_duplicates("customer_id_hashed")
df_both.shape

(23972063, 2)

In [55]:
df_both.to_csv("./BL_all_ids_shopped_in_18_or_signed_in_12_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [61]:
df_both=df_both[pd.notnull(df_both['email_address_hash'])]
df_both.shape

(21925777, 2)

In [62]:
df_both=df_both[~df_both['customer_id_hashed'].isin(df_all_Q3_control['customer_id_hashed'].tolist())]
df_both=df_both[~df_both['email_address_hash'].isin(df_all_Q3_control['email_address_hash'].tolist())]
df_both.shape

(21505499, 2)

In [64]:
df_both.to_csv("./BL_all_ids_shopped_in_18_or_signed_in_12_with_emails_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [68]:
import random
random.seed(1)
random_list=random.sample(range(len(df_both)), 100000)

df_both=df_both.reset_index()
del df_both['index']
df_both=df_both.reset_index()

df_both_control=df_both[df_both['index'].isin(random_list)]
df_both_test=df_both[~df_both['index'].isin(random_list)]


In [69]:
del df_both_control['index']
del df_both_test['index']
gc.collect()

268

In [70]:
df_both_control.shape

(100000, 2)

In [71]:
df_both_test.shape

(21405499, 2)

In [79]:
df_both_control_output=df_both_control[['email_address_hash']].rename(columns={"email_address_hash":"Email"}).drop_duplicates()
df_both_test_output=df_both_test[['email_address_hash']].rename(columns={"email_address_hash":"Email"}).drop_duplicates()


df_both_control_output.to_csv("./BL_FB_5_offer_control_emails_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
df_both_test_output.to_csv("./BL_FB_5_offer_test_emails_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [132]:
df_both_control_output.head(50)

,Email
135,9b954c51f5f4a7cf4a5df7409bd5d64cab56be9d3a0bd6...
312,a5dd3510fab6bab2fd29a8fece70f73b6700aeb5e9e98d...
621,aa61afbfc375a4d8fae0516ddf67ed8d6d4bcc0c2c55fe...
784,ce6595c1577a2218d270d4311ea700501a59946c5437e9...
839,44efe67dba35bb8e0fc73771f9a770fb8893ea3b89177a...
1286,008afd85e1a93cc6d375a9f70678d323eaebdf1ad31c22...
1657,c94eaa9110bf0315755e5b6851f68b67c78568ec9173f5...
1731,4e17227807f95766413e1806890ead3d7b85e29064d9e4...
1741,d0ea3b1eb2231789d94b53f96df79102ea3fdca7ae3105...
1845,e34f8870fd96f2eeb54e75ed03ea15bb694cf766822309...


In [80]:
df_both_control_output.shape,df_both_test_output.shape

((100000, 1), (21403357, 1))

In [81]:
df_both_control_output['Email'].nunique(),df_both_test_output['Email'].nunique()

(100000, 21403357)